**Challenge: Implement a Multiclass Classification Neural Network using PyTorch**

Objective:
Build a feedforward neural network using PyTorch to predict the species of iris flowers in a multiclass classification problem. The dataset used for this challenge is the Iris dataset, which consists of features like sepal length, sepal width, petal length, and petal width.

Steps:

1. **Data Preparation**: Load the MNIST dataset using ```torchvision.datasets.MNIST```. Standardize/normalize the features. Split the dataset into training and testing sets using, for example, ```sklearn.model_selection.train_test_split()```. **Bonus scores**: *use PyTorch's built-* ```DataLoader``` *to split the dataset*.

2. **Neural Network Architecture**: Define a simple feedforward neural network using PyTorch's ```nn.Module```. Design the input layer to match the number of features in the MNIST dataset and the output layer to have as many neurons as there are classes (10). You can experiment with the number of hidden layers and neurons to optimize the performance. **Bonus scores**: *Make your architecture flexibile to have as many hidden layers as the user wants, and use hyperparameter optimization to select the best number of hidden layeres.*

3. **Loss Function and Optimizer**: Choose an appropriate loss function for multiclass classification. Select an optimizer, like SGD (Stochastic Gradient Descent) or Adam.

4. **Training**: Write a training loop to iterate over the dataset.
Forward pass the input through the network, calculate the loss, and perform backpropagation. Update the weights of the network using the chosen optimizer.

5. **Testing**: Evaluate the trained model on the test set. Calculate the accuracy of the model.

6. **Optimization**: Experiment with hyperparameters (learning rate, number of epochs, etc.) to optimize the model's performance. Consider adjusting the neural network architecture for better results. **Notice that you can't use the optimization algorithms from scikit-learn that we saw in lab1: e.g.,** ```GridSearchCV```.


### 1. Data Preparation

In [38]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5), (0.5))
])

In [39]:
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

In [40]:
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)

In [41]:
print(trainset.data.shape)

torch.Size([60000, 28, 28])


In [69]:
batch_size =32

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, drop_last=True, shuffle=True)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, drop_last=True, shuffle=True)

### 2. Neural Network Architecture

In [87]:
import torch.nn as nn

class MNISTConvNet(nn.Module):

    def __init__(self, n_hiddenlayers, hiddenlayer_size):
        super().__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=28, kernel_size=(3,3), stride=1, padding=1)

        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout()

        self.conv2 = nn.Conv2d(in_channels=28, out_channels=28, kernel_size=(3,3), stride=1, padding=1)

        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))
        self.act2 = nn.ReLU()

        # (28, w, h)
        self.flat = nn.Flatten()

        self.fc3 = nn.Linear(5488, hiddenlayer_size)
        self.act3 = nn.ReLU()

        self.hidden_layers = []

        for layer in range(n_hiddenlayers):
            self.hidden_layers.append(nn.Linear(hiddenlayer_size, hiddenlayer_size))
            self.hidden_layers.append(nn.ReLU())


        self.fc4 = nn.Linear(hiddenlayer_size, 10)

    def forward(self, x):
        #input 1 x 28 x 28, output 28 x 28 x 28
        x = self.conv1(x)
        x = self.act1(x)
        #input 28 x 28 x 28, output 28 x 28 x 28
        x = self.conv2(x)
        x = self.act2(x)
        #input 28 x 28 x 28, output 28 x 14 x 14
        x = self.pool2(x)
        #input 28 x 14 x 14, 5488
        x = self.flat(x)
        x = self.fc3(x)
        x = self.act3(x)

        for element in self.hidden_layers:
            x = element(x)

        x = self.fc4(x)
        return x

Set 5 different models:
1. 2 hidden layers with size 300
2. 2 hidden layers with size 600
3. 2 hidden layers with size 1000
4. 3 hidden layers with size 600
5. 4 hidden layers with size 600

In [88]:
model1 = MNISTConvNet(2, 300)
model2 = MNISTConvNet(2, 600)
model3 = MNISTConvNet(2, 1000)
model4 = MNISTConvNet(3, 600)
model5 = MNISTConvNet(4, 600)

### 3. Loss Function and optimizer

In [ ]:
lr, momentum = 0.001, 0.9
loss_fn = nn.CrossEntropyLoss()
optimizer1 = torch.optim.SGD(model1.parameters(), lr=lr, momentum=momentum)
optimizer2 = torch.optim.SGD(model2.parameters(), lr=lr, momentum=momentum)
optimizer3 = torch.optim.SGD(model3.parameters(), lr=lr, momentum=momentum)
optimizer4 = torch.optim.SGD(model4.parameters(), lr=lr, momentum=momentum)
optimizer5 = torch.optim.SGD(model5.parameters(), lr=lr, momentum=momentum)
n_epochs = 5

### 4. / 5. / 6. Training and Testing, experimenting with different architectures and hyperparameters

In [97]:
# Train and compare the 5 different architectures
import numpy as np

# Run with 2 different learning rate, momentum combinations
hyperparameters = [[0.001, 0.01], [0.9, 0.9]]
for hp in range(2):
    # Run all the 5 models and evaluate them
    lr = hyperparameters[0][hp]
    momentum = hyperparameters[1][hp]
    print(f"Run with learning rate {lr} and momentum {momentum}")
    for epoch in range(n_epochs):
        losses1 = []
        losses2 = []
        losses3 = []
        losses4 = []
        losses5 = []

        for inputs, labels in trainloader:
            y_pred1 = model1(inputs)
            y_pred2 = model2(inputs)
            y_pred3 = model3(inputs)
            y_pred4 = model4(inputs)
            y_pred5 = model5(inputs)
            loss1 = loss_fn(y_pred1, labels)
            loss2 = loss_fn(y_pred2, labels)
            loss3 = loss_fn(y_pred3, labels)
            loss4 = loss_fn(y_pred4, labels)
            loss5 = loss_fn(y_pred5, labels)
            losses1.append(loss1)
            losses2.append(loss2)
            losses3.append(loss3)
            losses4.append(loss4)
            losses5.append(loss5)
            optimizer1.zero_grad()
            optimizer2.zero_grad()
            optimizer3.zero_grad()
            optimizer4.zero_grad()
            optimizer5.zero_grad()
            loss1.backward()
            loss2.backward()
            loss3.backward()
            loss4.backward()
            loss5.backward()
            optimizer1.step()
            optimizer2.step()
            optimizer3.step()
            optimizer4.step()
            optimizer5.step()

        print(f'Epoch {epoch + 1} --> loss model 1 = {sum(losses1)/len(losses1)}')
        print(f'Epoch {epoch + 1} --> loss model 2 = {sum(losses2)/len(losses2)}')
        print(f'Epoch {epoch + 1} --> loss model 3 = {sum(losses3)/len(losses3)}')
        print(f'Epoch {epoch + 1} --> loss model 4 = {sum(losses4)/len(losses4)}')
        print(f'Epoch {epoch + 1} --> loss model 5 = {sum(losses5)/len(losses5)}')

        acc = [0, 0, 0, 0, 0]
        count = [0, 0, 0, 0, 0]
        for inputs, labels in testloader:

            y_pred1 = model1(inputs)
            y_pred2 = model2(inputs)
            y_pred3 = model3(inputs)
            y_pred4 = model4(inputs)
            y_pred5 = model5(inputs)

            acc[0] += (torch.argmax(y_pred1, 1) == labels).float().sum()
            acc[1] += (torch.argmax(y_pred2, 1) == labels).float().sum()
            acc[2] += (torch.argmax(y_pred3, 1) == labels).float().sum()
            acc[3] += (torch.argmax(y_pred4, 1) == labels).float().sum()
            acc[4] += (torch.argmax(y_pred5, 1) == labels).float().sum()
            for i in range(len(count)):
                count[i] += len(labels)

        for i in range(len(acc)):
            acc[i] /= count[i]

        for i in range(len(acc)):
            print(f'Epoch {epoch + 1} --> model {i+1} accuracy = {acc[i]*100}')

Run with learning rate 0.001 and momentum 0.9
Epoch 1 --> loss model 1 = 0.03742362931370735
Epoch 1 --> loss model 2 = 0.04632783308625221
Epoch 1 --> loss model 3 = 0.03375035896897316
Epoch 1 --> loss model 4 = 0.08205492794513702
Epoch 1 --> loss model 5 = 0.12952156364917755
Epoch 1 --> model 1 accuracy = 98.50761413574219
Epoch 1 --> model 2 accuracy = 98.13702392578125
Epoch 1 --> model 3 accuracy = 98.54767608642578
Epoch 1 --> model 4 accuracy = 97.666259765625
Epoch 1 --> model 5 accuracy = 96.42427825927734
Epoch 2 --> loss model 1 = 0.033643849194049835
Epoch 2 --> loss model 2 = 0.04251237213611603
Epoch 2 --> loss model 3 = 0.029748613014817238
Epoch 2 --> loss model 4 = 0.07441126555204391
Epoch 2 --> loss model 5 = 0.12232403457164764
Epoch 2 --> model 1 accuracy = 98.48757934570312
Epoch 2 --> model 2 accuracy = 98.14703369140625
Epoch 2 --> model 3 accuracy = 98.2371826171875
Epoch 2 --> model 4 accuracy = 97.70632934570312
Epoch 2 --> model 5 accuracy = 96.1638641357

Unfortunately because of the long computing time only 5 epochs and very limited learning rate / momentum combinations were possible. In general the less layers the higher the initial accuracy was. The size did not really make a big difference. For more than two layers the learning rate of 0.001 was to small and a learning rate of 0.01 lead to faster convergence. However the models performed all very well and the differences were only marginal.